In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import *
import spacy
from spacy import displacy
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql.types import StringType
import collections
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

In [2]:
conf = SparkConf().setAppName("Yelp").setMaster("local[*]")
sc = SparkContext(conf=conf) 
sqlContext = SQLContext(sc)
df = sqlContext.read.json("../business.json")
nlp = spacy.load("en_core_web_sm")

In [3]:
#df.groupBy('categories').count().orderBy('count',ascending=False).show(truncate=False)

In [10]:
test= df.select('categories').distinct().head(50)
liste=list()
for x in test:
    liste.append(*x)

nlp = spacy.load("en_core_web_sm")
    
def word_tokenizer(text):
    #tokenizes and stems the text
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens if t not in spacy.lang.en.stop_words.STOP_WORDS]
    return tokens


def cluster_sentences(sentences, nb_of_clusters=5):
            tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenizer,
                                            stop_words=spacy.lang.en.stop_words.STOP_WORDS,
                                            max_df=0.9,
                                            min_df=0.1,
                                            lowercase=True)
            #builds a tf-idf matrix for the sentences
            tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
            kmeans = KMeans(n_clusters=nb_of_clusters)
            kmeans.fit(tfidf_matrix)
            clusters = collections.defaultdict(list)
            for i, label in enumerate(kmeans.labels_):
                    clusters[label].append(i)
            return dict(clusters)


if __name__ == "__main__":
    nbclusters = 6
    clusters = cluster_sentences(liste, nbclusters)
    for cluster in range(nbclusters):
        print ("cluster ",cluster,":")
        for i,sentence in enumerate(clusters[cluster]):
            print ("\t",liste[sentence], cluster)

C:\Users\Ashraf\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['d', 'll', 'm', 'n', 's', 't', 've', '‘', '’'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


cluster  0 :
	 Chocolatiers & Shops, Specialty Food, Ice Cream & Frozen Yogurt, Desserts, Food 0
	 Chicken Wings, Fast Food, Restaurants, Chicken Shop 0
	 Herbs & Spices, Convenience Stores, Food, Meat Shops, Specialty Food, Grocery, Do-It-Yourself Food 0
	 Convenience Stores, Food, Bakeries, Grocery 0
	 Coffee & Tea, Bakeries, Sandwiches, Restaurants, Food, Vegan, Desserts 0
	 Food, Fast Food, Restaurants, Sandwiches 0
	 Museums, Wineries, Food, Arts & Entertainment, Local Flavor 0
	 Cheese Shops, Delis, Sandwiches, Restaurants, Food, Specialty Food 0
cluster  1 :
	 Registration Services, Automotive 1
	 Car Wash, Automotive, Auto Detailing, Auto Repair 1
	 Bikes, Local Services, Bike Rentals, Sporting Goods, Bike Repair/Maintenance, Hotels & Travel, Tours, Mountain Biking, Active Life, Shopping 1
	 Automotive, Windshield Installation & Repair, Auto Glass Services, Body Shops 1
	 Electronics Repair, Arts & Entertainment, Shopping, Local Services, Casinos 1
	 Tires, Oil Change Stations,

In [11]:
print(clusters)

{0: [0, 3, 7, 10, 17, 20, 37, 48], 4: [1, 27, 34, 35, 43], 5: [2, 5, 6, 12, 16, 19, 24, 36, 42, 44], 2: [4, 11, 18, 38, 39, 49], 3: [8, 13, 14, 23, 32, 33, 40, 45], 1: [9, 15, 21, 22, 25, 26, 28, 29, 30, 31, 41, 46, 47]}
